In [1]:
import pandas as pd

In [2]:
#Read socioeconomic status per zipcode

In [3]:
socioeconomic_df = pd.read_csv("acs_socioeconomic_2019.csv")

In [4]:
#Read causal effect file(florida)

In [5]:
fl_treatment_effect = pd.read_csv('fl_climate_causal_effect.csv')

In [6]:
fl_treatment_effect.head()

,zip,1,2,3,4,5,6,elderlyReturns,returnsTotalwSalariesWages,returnsDependentCareCredit,returnsEducationCredit,risk_group,propensity_score,price_shift,y_0,y_0_corrected,y_1,y_1_corrected,treatment_effect,treatment_effect_no_corr
0,32003,0.203488,0.162064,0.132994,0.112645,0.267442,0.121366,0.340843,0.768895,0.273256,0.047965,True,0.450243,8.050216,12.372752,12.372752,9.591680,6.168053,-6.204699,-2.781072
1,32008,0.322314,0.268595,0.157025,0.099174,0.128099,0.024793,0.314050,0.780992,0.235537,0.037190,True,0.418107,12.456219,19.484131,19.484131,19.797981,2.238466,-17.245664,0.313850
2,32009,0.213836,0.220126,0.144654,0.144654,0.238994,0.037736,0.283019,0.830189,0.289308,0.031447,False,0.385439,14.755063,14.188574,15.110352,11.741358,11.741358,-3.368994,-2.447216
3,32011,0.232310,0.240320,0.157543,0.116155,0.212283,0.041389,0.257677,0.839786,0.295060,0.038718,True,0.383025,21.758099,15.149479,15.149479,13.147176,35.628516,20.479036,-2.002303
4,32024,0.295378,0.266065,0.158963,0.099211,0.146561,0.033822,0.299887,0.793687,0.262683,0.040586,True,0.407322,14.125399,18.123503,18.123503,17.729854,8.880704,-9.242799,-0.393649


In [7]:
#Merge by zip

In [8]:
socioeconomic_df['zip'] = socioeconomic_df['zip'].astype(str).str.zfill(5)
fl_treatment_effect['zip'] = fl_treatment_effect['zip'].astype(str).str.zfill(5)

In [9]:
fl_socioeconomic_treatment_effect = pd.merge(socioeconomic_df, fl_treatment_effect, on="zip")

In [10]:
#OLS regression treatment effect ~ socioeconomic variables

In [11]:
import statsmodels.api as sm

X = fl_socioeconomic_treatment_effect[['pct_poverty', 'pct_black', 'pct_college_plus', 'pct_mortgage']]
X = sm.add_constant(X)
y = fl_socioeconomic_treatment_effect['treatment_effect_no_corr']

model = sm.OLS(y, X).fit()
print(model.summary())

                               OLS Regression Results                               
Dep. Variable:     treatment_effect_no_corr   R-squared:                       0.765
Model:                                  OLS   Adj. R-squared:                  0.764
Method:                       Least Squares   F-statistic:                     632.5
Date:                      Thu, 08 May 2025   Prob (F-statistic):          1.81e-242
Time:                              20:40:45   Log-Likelihood:                -1053.6
No. Observations:                       781   AIC:                             2117.
Df Residuals:                           776   BIC:                             2141.
Df Model:                                 4                                         
Covariance Type:                  nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [12]:
# Mean treatment effect

In [13]:
fl_socioeconomic_treatment_effect['treatment_effect_no_corr'].mean()

-0.5514233676852156

In [14]:
# Mean treatment effect: Lower 25% & Upper 25%

In [15]:
low_poverty = fl_socioeconomic_treatment_effect[fl_socioeconomic_treatment_effect['pct_poverty'] <= fl_socioeconomic_treatment_effect['pct_poverty'].quantile(0.25)]

In [16]:
low_poverty['treatment_effect_no_corr'].mean()

-2.370608531544736

In [17]:
high_poverty = fl_socioeconomic_treatment_effect[fl_socioeconomic_treatment_effect['pct_poverty'] >= fl_socioeconomic_treatment_effect['pct_poverty'].quantile(0.75)]

In [18]:
high_poverty['treatment_effect_no_corr'].mean()

1.5595055765862325

In [19]:
low_black = fl_socioeconomic_treatment_effect[fl_socioeconomic_treatment_effect['pct_black'] <= fl_socioeconomic_treatment_effect['pct_black'].quantile(0.25)]

In [20]:
low_black['treatment_effect_no_corr'].mean()

-1.4976337368935495

In [21]:
high_black = fl_socioeconomic_treatment_effect[fl_socioeconomic_treatment_effect['pct_black'] >= fl_socioeconomic_treatment_effect['pct_black'].quantile(0.75)]

In [22]:
high_black['treatment_effect_no_corr'].mean()

1.012464626467932

In [23]:
low_college = fl_socioeconomic_treatment_effect[fl_socioeconomic_treatment_effect['pct_college_plus'] <= fl_socioeconomic_treatment_effect['pct_college_plus'].quantile(0.25)]

In [24]:
low_college['treatment_effect_no_corr'].mean()

1.1179150149421722

In [25]:
high_college = fl_socioeconomic_treatment_effect[fl_socioeconomic_treatment_effect['pct_college_plus'] >= fl_socioeconomic_treatment_effect['pct_college_plus'].quantile(0.75)]

In [26]:
high_college['treatment_effect_no_corr'].mean()

-2.5314638733610786

In [27]:
low_mortgage = fl_socioeconomic_treatment_effect[fl_socioeconomic_treatment_effect['pct_mortgage'] <= fl_socioeconomic_treatment_effect['pct_black'].quantile(0.25)]

In [28]:
low_mortgage['treatment_effect_no_corr'].mean()

-4.764395659193896

In [29]:
high_mortgage = fl_socioeconomic_treatment_effect[fl_socioeconomic_treatment_effect['pct_mortgage'] >= fl_socioeconomic_treatment_effect['pct_black'].quantile(0.75)]

In [30]:
high_mortgage['treatment_effect_no_corr'].mean()

-0.550066400623188

In [31]:
#Get Florida socioeconomic status

In [32]:
socioeconomic_df = pd.read_csv("acs_socioeconomic_2019.csv")

In [33]:
zipdata = pd.read_excel("uszips.xlsx")

In [34]:
socioeconomic_df['zip'] = socioeconomic_df['zip'].astype(str).str.zfill(5)
zipdata['zip'] = zipdata['zip'].astype(str).str.zfill(5)

In [35]:
florida_zipdata = zipdata[zipdata['state_name'] == 'Florida']

In [36]:
fl_socioeconomic = pd.merge(socioeconomic_df, florida_zipdata, on="zip")

In [37]:
fl_socioeconomic = fl_socioeconomic.dropna(subset=['pct_poverty', 'pct_black', 'pct_college_plus', 'pct_mortgage'])

In [38]:
#Cluster zip using socioeconomic variables

In [39]:
from sklearn.preprocessing import StandardScaler

In [40]:
features = ['pct_poverty', 'pct_black', 'pct_college_plus', 'pct_mortgage']

In [41]:
scaler = StandardScaler()
fl_socioeconomic_scaled = scaler.fit_transform(fl_socioeconomic[features])

In [42]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [43]:
#Clustering
range_n_clusters = [2, 3, 4, 5, 6, 7, 8, 9, 10]
for k in range_n_clusters: #try different numbers of clusters
    kmeans = KMeans(n_clusters=k, random_state=324)
    cluster_labels = kmeans.fit_predict(fl_socioeconomic_scaled)
    #report average Silhouette score
    silhouette_avg = silhouette_score(fl_socioeconomic_scaled, cluster_labels)
    print("For n_clusters ={},".format(k)+" the average silhouette_score is :{}".format(silhouette_avg))

C:\Users\user\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


For n_clusters =2, the average silhouette_score is :0.28188494182348955
For n_clusters =3, the average silhouette_score is :0.288891462475538
For n_clusters =4, the average silhouette_score is :0.2637167020805746


C:\Users\user\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\user\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\user\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


For n_clusters =5, the average silhouette_score is :0.25211646625789524
For n_clusters =6, the average silhouette_score is :0.24976272934352262
For n_clusters =7, the average silhouette_score is :0.23853598123920805


C:\Users\user\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\user\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\user\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


For n_clusters =8, the average silhouette_score is :0.235398120141123
For n_clusters =9, the average silhouette_score is :0.22579963200027856
For n_clusters =10, the average silhouette_score is :0.23711511446479283


C:\Users\user\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\user\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [44]:
kmeans = KMeans(n_clusters=3, random_state=324)
cluster_labels = kmeans.fit_predict(fl_socioeconomic_scaled)

C:\Users\user\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [45]:
fl_socioeconomic['cluster'] = cluster_labels

In [46]:
cluster_profile = fl_socioeconomic.groupby('cluster')[features].mean()
print(cluster_profile)

         pct_poverty  pct_black  pct_college_plus  pct_mortgage
cluster                                                        
0           0.165294   0.089504          0.174624      0.430360
1           0.263810   0.463085          0.181203      0.550507
2           0.093905   0.085134          0.384743      0.607713


In [47]:
fl_socioeconomic

,zip,pct_poverty,pct_black,pct_college_plus,pct_mortgage,lat,lng,city,state_id,state_name,...,density,county_fips,county_name,county_weights,county_names_all,county_fips_all,imprecise,military,timezone,cluster
0,32409,0.116979,0.005126,0.130453,0.584284,30.33447,-85.70516,Panama City,FL,Florida,...,33.4,12005,Bay,"{""12005"": 100}",Bay,12005,False,False,America/Chicago,0
1,32459,0.060926,0.007436,0.433720,0.705810,30.36527,-86.18429,Santa Rosa Beach,FL,Florida,...,156.6,12131,Walton,"{""12131"": 100}",Walton,12131,False,False,America/Chicago,2
2,32460,0.156743,0.288401,0.074032,0.283364,30.77565,-84.94949,Sneads,FL,Florida,...,28.1,12063,Jackson,"{""12063"": 100}",Jackson,12063,False,False,America/Chicago,0
3,32463,0.287037,0.106481,0.000000,0.264706,30.62790,-85.58728,Wausau,FL,Florida,...,168.3,12133,Washington,"{""12133"": 100}",Washington,12133,False,False,America/Chicago,0
4,32531,0.179226,0.018639,0.154731,0.473797,30.88513,-86.69600,Baker,FL,Florida,...,12.6,12091,Okaloosa,"{""12091"": 87.95, ""12113"": 12.05}",Okaloosa|Santa Rosa,12091|12113,False,False,America/Chicago,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
974,33028,0.043452,0.155542,0.487996,0.790983,26.01852,-80.34488,Pembroke Pines,FL,Florida,...,1880.2,12011,Broward,"{""12011"": 100}",Broward,12011,False,False,America/New_York,2
975,33037,0.159259,0.012851,0.337892,0.414542,25.19779,-80.38634,Key Largo,FL,Florida,...,160.4,12087,Monroe,"{""12087"": 100}",Monroe,12087,False,False,America/New_York,0
976,33054,0.348228,0.663742,0.083600,0.562867,25.90775,-80.25753,Opa Locka,FL,Florida,...,1432.7,12086,Miami-Dade,"{""12086"": 100}",Miami-Dade,12086,False,False,America/New_York,1
977,33060,0.252748,0.419068,0.196198,0.543996,26.23513,-80.12084,Pompano Beach,FL,Florida,...,1992.8,12011,Broward,"{""12011"": 100}",Broward,12011,False,False,America/New_York,1


In [48]:
fl_socioeconomic.to_csv("fl_socioeconomic.csv", index=False)